**GROUP BY 子句**

GROUP BY子句通过列或表达式的值将一组行分组为一个小分组的汇总行记录。 GROUP BY子句为每个分组返回一行。换句话说，它减少了结果集中的行数。

经常使用GROUP BY子句与聚合函数一起使用，如SUM，AVG，MAX，MIN和COUNT。SELECT子句中使用聚合函数来计算有关每个分组的信息。

GROUP BY子句是SELECT语句的可选子句。 下面是GROUP BY子句语法：   
SELECT    
    c1, c2,..., cn, aggregate_function(ci)   
FROM   
    table   
WHERE   
    where_conditions   
GROUP BY c1 , c2,...,cn;   

orders表进行示例，将订单状态的值分组到子组中，则要使用GROUP BY子句并指定按status列来执行分组，如下查询：   
**SELECT status FROM orders GROUP BY status;**   
GROUP BY子句返回状态(status)值是唯一的。它像DISTINCT运算符一样工作，如下面的查询所示：   
**SELECT DISTINCT status FROM orders;**

**GROUP BY 子句与聚合函数**

可使用聚合函数来执行一组行的计算并返回单个值。 GROUP BY子句通常与聚合函数一起使用以执行计算每个分组并返回单个值。   
例如，如果想知道每个状态中的订单数，可以使用COUNT函数与GROUP BY子句查询语句，如下所示：   
**SELECT status,COUNT(orderNumber) AS total FROM orders GROUP BY status;**

订单(orders)和订单详细(orderdetails)表，它们的ER图如下所示：![ER-orders-orderdetails](./images/ER-orders-orderdetails.png)   
要按状态获取所有订单的总金额，可以使用orderdetails表连接orders表，并使用SUM函数计算总金额。   
**SELECT status,SUM(quantityOrdered * priceEach) AS amount FROM orders INNER JOIN orderdetails USING (orderNumber) GROUP BY status;**

类似地，以下查询返回订单号和每个订单的总金额。    
**SELECT orderNumber, SUM(quantityOrdered * priceEach) AS total FROM orderdetails GROUP BY orderNumber;**

**GROUP BY 子句用表达式示例**

可以按表达式对行进行分组。以下查询获取每年的总销售额：    
**SELECT YEAR(orderDate) AS year,SUM(quantityOrdered * priceEach) AS  total FROM orders INNER JOIN orderdetails USING(orderNumber) GROUP BY year;**    
使用YEAR函数从订单日期(orderDate)中提取年份数据。只包括已发货(Shipped)状态的订单。 请注意，SELECT子句中出现的表达式必须与GROUP BY子句中的相同。

**GROUP BY与HAVING 子句**

可使用HAVING子句过滤GROUP BY子句返回的分组。以下查询使用HAVING子句来选择2013年以后的年销售总额。   
**SELECT YEAR(orderDate) AS year,SUM(quantityOrdered * priceEach) AS total FROM orders INNER JOIN orderdetails USING(orderNumber) WHERE status = 'Shipped' GROUP BY year HAVING year > 2013;**

**GROUP BY 子句：MySQL与标准SQL**

标准SQL不允许使用GROUP BY子句中的别名，但MySQL支持此选项。以下查询从订单日期提取年份，并对每年的订单进行计数。该year用作表达式YEAR(orderDate)的别名，它也用作GROUP BY子句中的别名，此查询在标准SQL中无效。    
**SELECT YEAR(orderDate) AS year, COUNT(orderNumber) FROM orders GROUP BY year;**

MySQL还允许您以升序或降序(标准SQL不能提供)对组进行排序。默认顺序是升序。例如，如果要按状态获取订单数量并按降序对状态进行排序，则可以使用带有DESC的GROUP BY子句，如下查询语句：   
**SELECT status, COUNT( * ) FROM  orders GROUP BY status DESC;**      
在GROUP BY子句中使用DESC以降序对状态进行排序。我们还可以在GROUP BY子句中明确指定ASC，按状态对分组进行升序排序。

**HAVING 子句**

在SELECT语句中使用HAVING子句来指定一组行或聚合的过滤条件。   
HAVING子句通常与GROUP BY子句一起使用，以根据指定的条件过滤分组。如果省略GROUP BY子句，则HAVING子句的行为与WHERE子句类似。   
请注意，HAVING子句将过滤条件应用于每组分行，而WHERE子句将过滤条件应用于每个单独的行。

使用orderdetails来示例，使用GROUP BY 子句来获取订单号，查看每个订单销售的商品数量和每个销售总额。   
**SELECT orderNumber,SUM(quantityOrdered) AS itemCount,SUM(quantityOrdered * priceEach) AS total FROM orderdetails GROUP BY orderNumber;**

可以通过使用HAVING子句查询(过滤)哪些订单的总销售额大于55000，如下所示：   
**SELECT orderNumber,SUM(quantityOrdered) AS itemCount,SUM(quantityOrdered * priceEach) AS total FROM orderdetails GROUP BY orderNumber HAVING total > 55000;**

可以使用逻辑运算符(如OR和AND)在HAVING子句中构造复杂过滤条件。 假设您想查找哪些订单的总销售额大于50000，并且包含超过600个项目，则可以使用以下查询：   
**SELECT orderNumber,SUM(quantityOrdered) AS itemCount,SUM(quantityOrdered * priceEach) AS total FROM orderdetails GROUP BY orderNumber HAVING total > 50000 AND itemCount >= 600;**

假设您想查找所有已发货(status='Shiped')的订单和总销售额大于55000的订单，可以使用INNER JOIN子句将orders表与orderdetails表一起使用，并在status列和总金额(total)列上应用条件，如以下查询所示：   
**SELECT orders.orderNumber,status,SUM(quantityOrdered * priceEach) AS total FROM orders INNER JOIN orderdetails USING(orderNumber) GROUP BY orders.orderNumber HAVING status = 'Shipped' AND total > 55000;**